In [ ]:
import os
import struct

### Use shebang to run the pip install commands on the console
### - fails when committing the kernel
#     ! python --version && pip --version
#     ! pip install --upgrade pip && pip --version

! python -m pip install nltk

### Use os to run the pip install commands using the os.system() method
### - works when committing the kernel
os.system("python --version && pip --version")
print("This version of Python is {} bits.".format(8 * struct.calcsize("P")))
os.system("pip install --upgrade pip && pip --version")

print("Installing / updating python packages using pip")
os.system("python -m pip install nltk")

# pylint: disable=C0413
import re
import math

from nltk.tokenize import word_tokenize, sent_tokenize


class SummariserTFIDFVariation:
    """ Summary text using the TFIDF method, does not remove stopwords"""


    # pylint: disable=R0201
    def remove_string_special_characters(self, sentence):
        """
            Removes special characters from within a string

            Parameters
            ==========
                s(str): single input string.

            Return
            ======
                stripped(str): A string with special characters removed
        """
        # Replace special character with ' '
        stripped = re.sub('[A\\w\\s]', '', sentence)
        stripped = re.sub('_', '', stripped)
        # Change any whitespace to one space
        stripped = re.sub('\\s+', '', stripped)
        # Remove start and end white spaces
        return stripped.strip()

    # pylint: disable=R0201
    def convert_sentences_into_documents(self, text_sentences):
        """
          Split the text into sentences and consider each sentence as a document,
          calculate the total word count of each sentence.

          Parameters
          ==========
          text_sentences:
            raw text to summarise, usually a long string of text made up of multiple sentences

          Return
          ======
          List of dictionary (doc id, words in the sentence) extracted from the sentence text
        """
        documents = []
        i = 0
        for sentence in text_sentences:
            i += 1
            num_of_words_in_the_sentence = len(word_tokenize(sentence))
            temp = {'doc_id' : i,
                    'doc_length' : num_of_words_in_the_sentence}

            documents.append(temp)
        return documents

    # pylint: disable=R0201
    def create_word_frequency_dictionary(self, sentences):
        """
          Create a frequency dictionary for each word in each document.

          Parameters
          ==========
          sentences:
            raw text to summarise, usually a long string of text made up of multiple sentences

          Return
          ======
          List of dictionary (doc id, dictionary of word frequency) extracted from the sentence text
        """
        i = 0
        word_frequency_dictionary = []
        for sentence in sentences:
            i += 1
            freq_dict = {}

            for word in word_tokenize(sentence):
                word = word.lower()
                if word in freq_dict:
                    freq_dict[word] += 1
                else:
                    freq_dict[word] = 1
                temp = {'doc_id' : i,
                        'freq_dict': freq_dict}

            word_frequency_dictionary.append(temp)

        return word_frequency_dictionary

    # pylint: disable=R0201
    def compute_document_tf_scores(self, documents, word_frequency_dictionaries):
        """
          tf = (frequency of the term in the doc
                  divided
                  by the total number of terms in the doc)

          Parameters
          ==========
          documents:
            texts stored as documents (with id and dictionary meta data)
          word_frequency_dictionaries:
            dictionary of word frequencies

          Return
          ======
          List of TF scores of the documents (based on the TF score calculation)
          along with the documents that are scored
        """
        scores = []
        for dictionary in word_frequency_dictionaries:
            doc_id = dictionary['doc_id']
            for k in dictionary['freq_dict']:
                frequency_of_terms_in_document = dictionary['freq_dict'][k]
                total_terms_in_the_doc = documents[doc_id-1]['doc_length']
                score = frequency_of_terms_in_document/total_terms_in_the_doc
                temp = {
                    'doc_id' : doc_id,
                    'TF_score' : score,
                    'key' : k
                }
                scores.append(temp)

        return scores

    def compute_document_idf_scores(self, documents, word_frequency_dictionary):
        """
          idf = ln(total number of docs divided by
                     number of docs with the term in it)
          ln() = natural log of

          Parameters
          ==========
          documents:
            texts stored as documents (with id and dictionary meta data)
          word_frequency_dictionaries:
            dictionary of word frequencies

          Return
          ======
          List of IDF scores of the documents (based on the IDF score calculation)
          along with the documents that are scored
        """
        scores = []
        counter = 0
        for dictionary in word_frequency_dictionary:
            counter += 1
            for k in list(dictionary['freq_dict'].keys()):
                total_documents = len(documents)
                total_docs_with_term_in_it = \
                     sum([k in dictionary['freq_dict'] \
                             for dictionary in word_frequency_dictionary])
                score = math.log(total_documents / total_docs_with_term_in_it)

                temp = {'doc_id' : counter,
                        'IDF_score' : score,
                        'key' : k}

                scores.append(temp)

        return scores

    def compute_document_tfidf_scores(self, tf_scores, idf_scores):
        """
          computes the product of TF and IDF scores
          for each document (sentence) for documents that match
          by key and doc_id

          Parameters
          ==========
          tf_scores:
            TF scores of the sentences of the text to summarise
          idf_scores:
            IDF scores of the sentences of the text to summarise

          Return
          ======
          List of TFIDF scores of the documents (based on the TFIDF score calculation)
          along with the documents that are scored
        """
        scores = []
        for j in idf_scores:
            for i in tf_scores:
                if j['key'] == i['key'] and j['doc_id'] == i['doc_id']:
                    temp = {'doc_id' : j['doc_id'],
                            'TFIDF_score' : i['TF_score'] * j['IDF_score'],
                            'key' : i['key']}
            scores.append(temp)
        return scores

    def compute_sentence_scores(self, tfidf_scores, sentences, documents):
        """
          The score of a sentence is calculated by adding
          the TFIDF scores of the words that make up the sentence for
          the document that represents that sentence

          Parameters
          ==========
          tfidf_scores:
            TFIF scores of the sentences of the text to summarise
          sentences:
            Raw sentences of the text to summarise
          documents:
            Raw sentences as documents of the text to summarise

          Return
          ======
          List of dictionary of sentences with their scores and doc id
        """

        sentence_info = []
        for document in documents:
            # pylint: disable=W0105
            """Loop through each document(sentence) and calculates their 'score'"""
            score = 0
            # pylint: disable=W0612
            for index, value in enumerate(tfidf_scores):
                dictionary = value
                if document['doc_id'] == dictionary['doc_id']:
                    score += dictionary['TFIDF_score']

            temp = {'doc_id' : document['doc_id'],
                    'sent_score' : score,
                    'sentence' : sentences[document['doc_id'] - 1]}

            sentence_info.append(temp)

        return sentence_info

    def filter_sentences_by_score(self, sentence_info, top_n_sentences):
        """
          Filter sentences after summing their scores,
          only select sentences that meet a criteria.
          Types of criteria:
             - top n sentences (scores sorted in descending order)
             - sentence score >= average
                    --- suitable for average or smaller sized documents
             - sentence score >= average + (1.5 * stddev)
                    --- for medium sized documents
             - sentence score >= average + (3 * stddev)
                    --- for large documents

          The current functionality checks for the first type of condition.
          These sentences joined together make up the summary.

          Parameters
          ==========
          sentence_info:
            list of dictionary of sentences with their scores
          top_n_sentences:
            number of sentences to pick from the top scored sentences (descending order)

          Return
          ======
          List of top n sentences from the scored list of sentences that form the summary
          and also the whole list of sorted sentences
        """
        summary = []

        # pylint: disable=W0105
        """
            Sort the sentences by its score in descending order
            and pick the top n sentences from the list
        """
        sorted_sentence_info = sorted(sentence_info, key=lambda k: -k['sent_score'])
        counter = 0
        if top_n_sentences > len(sorted_sentence_info):
            top_n_sentences = len(sorted_sentence_info)

        for sentence in sorted_sentence_info:
            if counter < top_n_sentences:
                summary.append(sentence['sentence'])
                counter = counter + 1
            else:
                break

        return ' '.join(summary), sorted_sentence_info

    def generate_summary(self, text, top_n_sentences):
        """
          Get the text, clean it by removing special characters,
          get the frequency of the words and create a dictionary,
          compute the TF scores of sentences,
          compute the IDF scores of sentences,
          compute the combined TF-IDF scores of sentences,

          Iterate through sentences to create a summary based on a threshold point.
          Sentences with scores less than that don't make the summary.

          Parameters
          ==========
          text:
            raw text that needs summarisation
          top_n_sentences:
            number of sentences to pick from the top scored sentences (descending order)

          Return
          ======
          List of top n sentences from the scored list of sentences that form the summary
          and also the whole list of sorted sentences
        """
        tokenised_text_sentences = sent_tokenize(text)
        clean_tokenised_text_sentences = [
            self.remove_string_special_characters(sentence) for sentence in tokenised_text_sentences
        ]
        documents = self.convert_sentences_into_documents(clean_tokenised_text_sentences)

        word_frequency_dictionaries = \
            self.create_word_frequency_dictionary(clean_tokenised_text_sentences)
        document_tf_scores = \
            self.compute_document_tf_scores(documents, word_frequency_dictionaries)
        document_idf_scores = \
            self.compute_document_idf_scores(documents, word_frequency_dictionaries)

        document_tfidf_scores = \
            self.compute_document_tfidf_scores(document_tf_scores, document_idf_scores)
        sentences_info = \
            self.compute_sentence_scores(document_tfidf_scores, tokenised_text_sentences, documents)
        summarised_text, sorted_sentence_info = \
            self.filter_sentences_by_score(sentences_info, top_n_sentences)
        return summarised_text, sorted_sentence_info

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
This version of Python is 64 bits.
Installing / updating python packages using pip


In [ ]:
stringT = """WhatsApp Privacy Policy
If you live in the European Region, WhatsApp Ireland Limited provides the services to you under this Terms of Service and Privacy Policy.

WhatsApp Legal Info
If you live outside the European Region, WhatsApp LLC ("WhatsApp," "our," "we," or "us") provides our Services to you under this Terms of Service and Privacy Policy.

Our Privacy Policy ("Privacy Policy") helps explain our data practices, including the information we process to provide our Services.

For example, our Privacy Policy talks about what information we collect and how this affects you. It also explains the steps we take to protect your privacy, like building our Services so delivered messages aren’t stored by us and giving you control over who you communicate with on our Services.

We are one of the Meta Companies. You can learn more further below in this Privacy Policy about the ways in which we share information across this family of companies.

This Privacy Policy applies to all of our Services unless specified otherwise.

Please also read WhatsApp’s Terms of Service ("Terms"), which describe the terms under which you use and we provide our Services.

Back to top

Key Updates
Respect for your privacy is coded into our DNA. Since we started WhatsApp, we’ve built our services with a set of strong privacy principles in mind. In our updated Terms of Service and Privacy Policy you’ll find:

Additional Information On How We Handle Your Data. Our updated Terms and Privacy Policy provide more information on how we process your data, and our commitment to privacy. For example, we’ve added more information about more recent product features and functionalities, how we process your data for safety, security, and integrity, and added more direct links to user settings, Help Center articles and how you can manage your information.
Better Communication With Businesses. Many businesses rely on WhatsApp to communicate with their customers and clients. We work with businesses that use Meta or third parties to help store and better manage their communications with you on WhatsApp.
Making It Easier To Connect. As part of the Meta Companies, WhatsApp partners with Meta to offer experiences and integrations across Meta’s family of apps and products.
Back to top

Information We Collect
WhatsApp must receive or collect some information to operate, provide, improve, understand, customize, support, and market our Services, including when you install, access, or use our Services.

The types of information we receive and collect depend on how you use our Services. We require certain information to deliver our Services and without this we will not be able to provide our Services to you. For example, you must provide your mobile phone number to create an account to use our Services.

Our Services have optional features which, if used by you, require us to collect additional information to provide such features. You will be notified of such collection, as appropriate. If you choose not to provide the information needed to use a feature, you will be unable to use the feature. For example, you cannot share your location with your contacts if you do not permit us to collect your location data from your device. Permissions can be managed through your Settings menu on both Android and iOS devices.

Information You Provide
Your Account Information. You must provide your mobile phone number and basic information (including a profile name of your choice) to create a WhatsApp account. If you don’t provide us with this information, you will not be able to create an account to use our Services. You can add other information to your account, such as a profile picture and "about" information.
Your Messages. We do not retain your messages in the ordinary course of providing our Services to you. Instead, your messages are stored on your device and not typically stored on our servers. Once your messages are delivered, they are deleted from our servers. The following scenarios describe circumstances where we may store your messages in the course of delivering them:
Undelivered Messages. If a message cannot be delivered immediately (for example, if the recipient is offline), we keep it in encrypted form on our servers for up to 30 days as we try to deliver it. If a message is still undelivered after 30 days, we delete it.
Media Forwarding. When a user forwards media within a message, we store that media temporarily in encrypted form on our servers to aid in more efficient delivery of additional forwards.

We offer end-to-end encryption for our Services. End-to-end encryption means that your messages are encrypted to protect against us and third parties from reading them. Learn more about end-to-end encryption and how businesses communicate with you on WhatsApp.
Your Connections. You can use the contact upload feature and provide us, if permitted by applicable laws, with the phone numbers in your address book on a regular basis, including those of users of our Services and your other contacts. If any of your contacts aren’t yet using our Services, we’ll manage this information for you in a way that ensures those contacts cannot be identified by us. Learn more about our contact upload feature here. You can create, join, or get added to groups and broadcast lists, and such groups and lists get associated with your account information. You give your groups a name. You can provide a group profile picture or description.
Status Information. You may provide us your status if you choose to include one on your account. Learn how to use status on Android, iPhone, or KaiOS.
Transactions And Payments Data. If you use our payments services, or use our Services meant for purchases or other financial transactions, we process additional information about you, including payment account and transaction information. Payment account and transaction information includes information needed to complete the transaction (for example, information about your payment method, shipping details and transaction amount). If you use our payments services available in your country or territory, our privacy practices are described in the applicable payments privacy policy.
Customer Support And Other Communications. When you contact us for customer support or otherwise communicate with us, you may provide us with information related to your use of our Services, including copies of your messages, any other information you deem helpful, and how to contact you (e.g., an email address). For example, you may send us an email with information relating to app performance or other issues.

Automatically Collected Information
Usage And Log Information. We collect information about your activity on our Services, like service-related, diagnostic, and performance information. This includes information about your activity (including how you use our Services, your Services settings, how you interact with others using our Services (including when you interact with a business), and the time, frequency, and duration of your activities and interactions), log files, and diagnostic, crash, website, and performance logs and reports. This also includes information about when you registered to use our Services; the features you use like our messaging, calling, Status, groups (including group name, group picture, group description), payments or business features; profile photo, "about" information; whether you are online, when you last used our Services (your "last seen"); and when you last updated your "about" information.
Device And Connection Information. We collect device and connection-specific information when you install, access, or use our Services. This includes information such as hardware model, operating system information, battery level, signal strength, app version, browser information, mobile network, connection information (including phone number, mobile operator or ISP), language and time zone, IP address, device operations information, and identifiers (including identifiers unique to Meta Company Products associated with the same device or account).
Location Information. We collect and use precise location information from your device with your permission when you choose to use location-related features, like when you decide to share your location with your contacts or view locations nearby or locations others have shared with you. There are certain settings relating to location-related information which you can find in your device settings or the in-app settings, such as location sharing. Even if you do not use our location-related features, we use IP addresses and other information like phone number area codes to estimate your general location (e.g., city and country). We also use your location information for diagnostics and troubleshooting purposes.
Cookies. We use cookies to operate and provide our Services, including to provide our Services that are web-based, improve your experiences, understand how our Services are being used, and customize them. For example, we use cookies to provide our Services for web and desktop and other web-based services. We may also use cookies to understand which of our Help Center articles are most popular and to show you relevant content related to our Services. Additionally, we may use cookies to remember your choices, like your language preferences, to provide a safer experience, and otherwise to customize our Services for you. Learn more about how we use cookies to provide you our Services.

Back to top

Third-Party Information
Information Others Provide About You. We receive information about you from other users. For example, when other users you know use our Services, they may provide your phone number, name, and other information (like information from their mobile address book) just as you may provide theirs. They may also send you messages, send messages to groups to which you belong, or call you. We require each of these users to have lawful rights to collect, use, and share your information before providing any information to us.
You should keep in mind that in general any user can capture screenshots of your chats or messages or make recordings of your calls with them and send them to WhatsApp or anyone else, or post them on another platform.
User Reports. Just as you can report other users, other users or third parties may also choose to report to us your interactions and your messages with them or others on our Services; for example, to report possible violations of our Terms or policies. When a report is made, we collect information on both the reporting user and reported user. To find out more about what happens when a user report is made, please see Advanced Safety and Security Features.
Businesses On WhatsApp. Businesses you interact with using our Services may provide us with information about their interactions with you. We require each of these businesses to act in accordance with applicable law when providing any information to us.
When you message with a business on WhatsApp, keep in mind that the content you share may be visible to several people in that business. In addition, some businesses might be working with third-party service providers (which may include Meta) to help manage their communications with their customers. For example, a business may give such third-party service provider access to its communications to send, store, read, manage, or otherwise process them for the business. To understand how a business processes your information, including how it might share your information with third parties or Meta, you should review that business’ privacy policy or contact the business directly.
Third-Party Service Providers. We work with third-party service providers and other Meta Companies to help us operate, provide, improve, understand, customize, support, and market our Services. For example, we work with them to distribute our apps; provide our technical and physical infrastructure, delivery, and other systems; provide engineering support, cybersecurity support, and operational support; supply location, map, and places information; process payments; help us understand how people use our Services; market our Services; help you connect with businesses using our Services; conduct surveys and research for us; ensure safety, security, and integrity; and help with customer service. These companies may provide us with information about you in certain circumstances; for example, app stores may provide us with reports to help us diagnose and fix service issues.
The "How We Work With Other Meta Companies" section below provides more information about how WhatsApp collects and shares information with the other Meta Companies.
Third-Party Services. We allow you to use our Services in connection with third-party services and Meta Company Products. If you use our Services with such third-party services or Meta Company Products, we may receive information about you from them; for example, if you use the WhatsApp share button on a news service to share a news article with your WhatsApp contacts, groups, or broadcast lists on our Services, or if you choose to access our Services through a mobile carrier’s or device provider’s promotion of our Services. Please note that when you use third-party services or Meta Company Products, their own terms and privacy policies will govern your use of those services and products.
Back to top

How We Use Information
We use information we have (subject to choices you make and applicable law) to operate, provide, improve, understand, customize, support, and market our Services. Here's how:

Our Services. We use information we have to operate and provide our Services, including providing customer support; completing purchases or transactions; improving, fixing, and customizing our Services; and connecting our Services with Meta Company Products that you may use. We also use information we have to understand how people use our Services; evaluate and improve our Services; research, develop, and test new services and features; and conduct troubleshooting activities. We also use your information to respond to you when you contact us.
Safety, Security, And Integrity. Safety, security, and integrity are an integral part of our Services. We use information we have to verify accounts and activity; combat harmful conduct; protect users against bad experiences and spam; and promote safety, security, and integrity on and off our Services, such as by investigating suspicious activity or violations of our Terms and policies, and to ensure our Services are being used legally. Please see the Law, Our Rights and Protection section below for more information.
Communications About Our Services And The Meta Companies. We use information we have to communicate with you about our Services and let you know about our terms, policies, and other important updates. We may provide you marketing for our Services and those of the Meta Companies.
No Third-Party Banner Ads. We still do not allow third-party banner ads on our Services. We have no intention to introduce them, but if we ever do, we will update this Privacy Policy.
Business Interactions. We enable you and third parties, like businesses, to communicate and interact with each other using our Services, such as Catalogs for businesses on WhatsApp through which you can browse products and services and place orders. Businesses may send you transaction, appointment, and shipping notifications; product and service updates; and marketing. For example, you may receive flight status information for upcoming travel, a receipt for something you purchased, or a notification when a delivery will be made. Messages you receive from a business could include an offer for something that might interest you. We do not want you to have a spammy experience; as with all of your messages, you can manage these communications, and we will honor the choices you make.
Back to top

Information You And We Share
You share your information as you use and communicate through our Services, and we share your information to help us operate, provide, improve, understand, customize, support, and market our Services.

Send Your Information To Those With Whom You Choose To Communicate. You share your information (including messages) as you use and communicate through our Services.
Information Associated With Your Account. Your phone number, profile name and photo, "about" information, last seen information, and message receipts are available to anyone who uses our Services, although you can configure your Services settings to manage certain information available to other users, including businesses, with whom you communicate.
Your Contacts And Others. Users, including businesses, with whom you communicate can store or reshare your information (including your phone number or messages) with others on and off our Services. You can use your Services settings and the “block” feature in our Services to manage who you communicate with on our Services and certain information you share.
Businesses On WhatsApp. We offer specific services to businesses such as providing them with metrics regarding their use of our Services.
Third-Party Service Providers. We work with third-party service providers and other Meta Companies to help us operate, provide, improve, understand, customize, support, and market our Services. We work with these companies to support our Services, such as to provide technical infrastructure, delivery and other systems; market our Services; conduct surveys and research for us; protect the safety, security, and integrity of users and others; and assist with customer service. When we share information with third-party service providers and other Meta Companies in this capacity, we require them to use your information on our behalf in accordance with our instructions and terms.
Third-Party Services. When you or others use third-party services or other Meta Company Products that are integrated with our Services, those third-party services may receive information about what you or others share with them. For example, if you use a data backup service integrated with our Services (like iCloud or Google Drive), they will receive information you share with them, such as your WhatsApp messages. If you interact with a third-party service or another Meta Company Product linked through our Services, such as when you use the in-app player to play content from a third-party platform, information about you, like your IP address and the fact that you are a WhatsApp user, may be provided to such third party or Meta Company Product. Please note that when you use third-party services or other Meta Company Products, their own terms and privacy policies will govern your use of those services and products.
Back to top

How We Work With Other Meta Companies
As part of the Meta Companies, WhatsApp receives information from, and shares information (see here) with, the other Meta Companies. We may use the information we receive from them, and they may use the information we share with them, to help operate, provide, improve, understand, customize, support, and market our Services and their offerings, including the Meta Company Products. This includes:

helping improve infrastructure and delivery systems;
understanding how our Services or theirs are used;
promoting safety, security, and integrity across the Meta Company Products, e.g., securing systems and fighting spam, threats, abuse, or infringement activities;
improving their services and your experiences using them, such as making suggestions for you (for example, of friends or group connections, or of interesting content), personalizing features and content, helping you complete purchases and transactions, and showing relevant offers and ads across the Meta Company Products; and
providing integrations which enable you to connect your WhatsApp experiences with other Meta Company Products. For example, allowing you to connect your Facebook Pay account to pay for things on WhatsApp or enabling you to chat with your friends on other Meta Company Products, such as Portal, by connecting your WhatsApp account.
Learn more about the other Meta Companies and their privacy practices by reviewing their privacy policies.

Back to top

Assignment, Change Of Control, And Transfer
In the event that we are involved in a merger, acquisition, restructuring, bankruptcy, or sale of all or some of our assets, we will share your information with the successor entities or new owners in connection with the transaction in accordance with applicable data protection laws.

Back to top

Managing And Retaining Your Information
You can access or port your information using our in-app Request Account Info feature (available under Settings > Account). For iPhone users, you can learn how to access, manage, and delete your information through our iPhone Help Center articles. For Android users, you can learn how to access, manage, and delete your information through our Android Help Center articles.

We store information for as long as necessary for the purposes identified in this Privacy Policy, including to provide our Services or for other legitimate purposes, such as complying with legal obligations, enforcing and preventing violations of our Terms, or protecting or defending our rights, property, and users. The storage periods are determined on a case-by-case basis that depends on factors like the nature of the information, why it is collected and processed, relevant legal or operational retention needs, and legal obligations.

If you would like to further manage, change, limit, or delete your information, you can do that through the following tools:

Services Settings. You can change your Services settings to manage certain information available to other users. You can manage your contacts, groups, and broadcast lists, or use our “block” feature to manage the users with whom you communicate.
Changing Your Mobile Phone Number, Profile Name And Picture, And “About” Information. If you change your mobile phone number, you must update it using our in-app change number feature and transfer your account to your new mobile phone number. You can also change your profile name, profile picture, and "about" information at any time.
Deleting Your WhatsApp Account. You can delete your WhatsApp account at any time (including if you want to revoke your consent to our use of your information pursuant to applicable law) using our in-app delete my account feature. When you delete your WhatsApp account, your undelivered messages are deleted from our servers as well as any of your other information we no longer need to operate and provide our Services. Deleting your account will, for example, delete your account info and profile photo, delete you from all WhatsApp groups, and delete your WhatsApp message history. Be mindful that if you only delete WhatsApp from your device without using our in-app delete my account feature, your information will be stored with us for a longer period. Please remember that when you delete your account, it does not affect your information related to the groups you created or the information other users have relating to you, such as their copy of the messages you sent them.
You can learn more here about our data deletion and retention practices and about how to delete your account.

Back to top

Law, Our Rights, And Protection
We access, preserve, and share your information described in the "Information We Collect" section of this Privacy Policy above if we have a good-faith belief that it is necessary to: (a) respond pursuant to applicable law or regulations, legal process, or government requests; (b) enforce our Terms and any other applicable terms and policies, including for investigations of potential violations; (c) detect, investigate, prevent, or address fraud and other illegal activity or security, and technical issues; or (d) protect the rights, property, and safety of our users, WhatsApp, the other Meta Companies, or others, including to prevent death or imminent bodily harm.

Back to top

Our Global Operations
WhatsApp shares information globally, both internally within the Meta Companies and externally with our partners and service providers, and with those with whom you communicate around the world, in accordance with this Privacy Policy. Your information may, for example, be transferred or transmitted to, or stored and processed in, the United States; countries or territories where the Meta Companies’ affiliates and partners, or our service providers are located; or any other country or territory globally where our Services are provided outside of where you live for the purposes as described in this Privacy Policy. WhatsApp uses Meta’s global infrastructure and data centers, including in the United States. These transfers are necessary to provide the global Services set forth in our Terms. Please keep in mind that the countries or territories to which your information is transferred may have different privacy laws and protections than what you have in your home country or territory.

Back to top

Updates To Our Policy
We may amend or update our Privacy Policy. We will provide you notice of amendments to this Privacy Policy, as appropriate, and update the “Last modified” date at the top of this Privacy Policy. Please review our Privacy Policy from time to time.

Back to top



Contact Us

If you have questions or issues about our Privacy Policy, please contact us.

WhatsApp LLC

Privacy Policy

1601 Willow Road

Menlo Park, California 94025

United States of America"""

In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
sam = SummariserTFIDFVariation() 
textS = stringT

x = sam.generate_summary(stringT,50)

print(x[0])

Here's how:

Our Services. You can use your Services settings and the “block” feature in our Services to manage who you communicate with on our Services and certain information you share. The "How We Work With Other Meta Companies" section below provides more information about how WhatsApp collects and shares information with the other Meta Companies. Back to top

Managing And Retaining Your Information
You can access or port your information using our in-app Request Account Info feature (available under Settings > Account). WhatsApp Legal Info
If you live outside the European Region, WhatsApp LLC ("WhatsApp," "our," "we," or "us") provides our Services to you under this Terms of Service and Privacy Policy. In our updated Terms of Service and Privacy Policy you’ll find:

Additional Information On How We Handle Your Data. Please also read WhatsApp’s Terms of Service ("Terms"), which describe the terms under which you use and we provide our Services. Our Privacy Policy ("Privacy Policy")